# 주문지역 별 집계

In [10]:
import pandas as pd
import numpy as np

# df = pd.read_csv('./datas/klm1144__dataframe.csv')
curr_df = pd.read_csv('2020-9/klm1144.csv')
compare_df = pd.read_csv('2020-11/klm1144.csv')

In [11]:
curr_order = curr_df.groupby('address').sum('payment')
compare_order = compare_df.groupby('address').sum('payment')

In [12]:
curr_order['profit'] = curr_order['payment'] - compare_order['payment']
curr_order.loc[:, ~curr_order.columns.str.contains('^Unnamed')]
curr_order

,Unnamed: 0,campaign_id,payment,profit
address,,,,
경기 수원시 권선구 서둔동 ****,109,3046829,12000,-227000.0
경기 수원시 권선구 세류2동 ****,35,3046829,10000,NaN
경기 수원시 권선구 평동 ****,9623,314247400,2206000,-1115000.0
경기 화성시 매송면 매송면 ****,4658,135233820,934000,286000.0
경기 화성시 봉담읍 봉담읍 ****,1528,63109137,480000,-701000.0
경기도 수원시 권선구 평동 ****,233,6052026,32000,16000.0


In [13]:
for address in curr_order[curr_order.profit.isna()].index.to_list():
    if address in curr_order.index:
        curr_order.loc[address] = curr_order.loc[address].payment
    elif address in compare_order.index:
        curr_order.loc[address] = -compare_order.loc[address].payment
    else:
        raise ValueError('뭔가 잘못 되었다.')

In [14]:
curr_order

,Unnamed: 0,campaign_id,payment,profit
address,,,,
경기 수원시 권선구 서둔동 ****,109.0,3046829.0,12000.0,-227000.0
경기 수원시 권선구 세류2동 ****,10000.0,10000.0,10000.0,10000.0
경기 수원시 권선구 평동 ****,9623.0,314247400.0,2206000.0,-1115000.0
경기 화성시 매송면 매송면 ****,4658.0,135233820.0,934000.0,286000.0
경기 화성시 봉담읍 봉담읍 ****,1528.0,63109137.0,480000.0,-701000.0
경기도 수원시 권선구 평동 ****,233.0,6052026.0,32000.0,16000.0


# 캠페인 아이디 별 집계

In [21]:
import json

with open("2020-11/adv_info.json", "r") as f:
    adv_info = json.load(f)
adv_info

{'3005196': '경기도 화성시 매송면 화성로2397번길 49',
 '3005197': '경기도 수원시 권선구 삼천병마로 1545',
 '3046829': '경기도 수원시 권선구 산업로 174-30',
 '3251512': '경기도 수원시 권선구 매송고색로794번길 249'}

In [30]:
curr_order = curr_df.groupby('campaign_id').sum('payment')
curr_order.index = curr_order.index.map(str)
curr_order = curr_order.loc[:, ~curr_order.columns.str.contains('^Unnamed')]
curr_order.profit= np.nan

compare_order = compare_df.groupby('campaign_id').sum('payment')
compare_order = compare_order.loc[:, ~compare_order.columns.str.contains('^Unnamed')]
compare_order.index = compare_order.index.map(lambda x: str(x))
compare_order.profit = np.nan

Index(['3005196', '3005197', '3046829'], dtype='object', name='campaign_id')

In [39]:
try:
    curr_order = curr_order.drop('기타')
except KeyError:
    pass
try:
    compare_order = compare_order.drop('기타')
except KeyError:
    pass

In [50]:
curr_order['profit'] = (curr_order.payment - compare_order.payment - 88000)
curr_order['profit'] = curr_order['profit'].fillna(curr_order['payment'] - 88000) # 이전달에 울트라콜이 있고 이번달에 없을 수 없음. 무조건 현재달 기준.
curr_order['profit'] = curr_order['profit'] / compare_order.payment * 100 

In [47]:
curr_order.payment - compare_order.payment - 88000 # 증가량

campaign_id
3005196    268000.0
3005197   -963000.0
3046829   -977000.0
3251512         NaN
Name: payment, dtype: float64

In [20]:
compare_order

,payment
campaign_id,
3005196,654000
3005197,2733000
3046829,1881000
3251512,1352000
